In [39]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing

# Random Forest 

In [40]:
df = pd.read_csv("train.csv")
df1 = pd.read_csv("test.csv")

In [41]:
df['agregados']= df['escuelascercanas']+df['gimnasio']+df['centroscomercialescercanos']+df['piscina']+df['usosmultiples']
train = df.drop(['id','idzona','lat','direccion','ciudad','descripcion','titulo','lng', 'escuelascercanas','gimnasio','centroscomercialescercanos','piscina','usosmultiples'],1)
train['fecha'] = pd.to_datetime(train['fecha']).dt.year
df1['agregados']= df1['escuelascercanas']+df1['gimnasio']+df1['centroscomercialescercanos']+df1['piscina']+df1['usosmultiples']
test = df1.drop(['id','idzona','lat','direccion','ciudad','descripcion','titulo','lng', 'escuelascercanas','gimnasio','centroscomercialescercanos','piscina','usosmultiples'],1)
test['fecha'] = pd.to_datetime(test['fecha']).dt.year
#no acepta nan
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [42]:
X = train
y = train.iloc[:, 8].values 

In [43]:
y_train = train['precio']
temporal = train

In [44]:
def mean_target_encoding(train, nombrefeat, nombretarget):
        
    temp = train.groupby(nombrefeat)[nombretarget].transform(np.mean) #Saco promedio
    temp = preprocessing.scale(temp) #Normalizo
    train[nombrefeat + "_mean"] = temp #Dejo en el DF
    
    return train

In [45]:
def mean_target_decoding(x_test, nombrefeat, x_train):
    
    nombrefeatmean = nombrefeat + "_mean"
    
    temp = x_train.loc[:,[nombrefeat,nombrefeatmean]]
    temp = temp.set_index(nombrefeat)
    temp = temp.drop_duplicates()
    temp = temp.T.squeeze()
    values = x_test[nombrefeat].map(temp)
    x_test[nombrefeatmean] = values 

    return x_test

In [46]:
#Calculo mean para provincia y tipo de propiedad 
x_train = mean_target_encoding(train, "provincia", "precio")
x_train['precio'] = y_train
x_train = mean_target_encoding(x_train, "tipodepropiedad", "precio")
test = mean_target_decoding(test, "provincia", temporal)
test = mean_target_decoding(test, "tipodepropiedad", temporal)

In [31]:
#limpio los mean
x_train.drop(["precio","provincia", "tipodepropiedad"], axis=1, inplace = True)
test.drop(["provincia", "tipodepropiedad"], axis=1, inplace = True)

In [32]:
test

,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,fecha,agregados,provincia_mean,tipodepropiedad_mean
0,29.0,3.0,0.0,4.0,300.0,0.0,2013,0.0,1.374282,-0.407201
1,0.0,1.0,1.0,1.0,67.0,67.0,2015,0.0,-0.722759,0.714871
2,0.0,2.0,1.0,2.0,87.0,100.0,2015,1.0,1.374282,0.714871
3,2.0,2.0,2.0,2.0,86.0,86.0,2015,0.0,0.026151,0.714871
4,10.0,2.0,1.0,1.0,80.0,76.0,2013,2.0,0.338326,-0.407201
...,...,...,...,...,...,...,...,...,...,...
59995,20.0,4.0,3.0,3.0,291.0,0.0,2015,2.0,1.374282,-0.407201
59996,10.0,3.0,1.0,2.0,71.0,87.0,2016,2.0,0.338326,-0.407201
59997,5.0,3.0,2.0,2.0,102.0,0.0,2014,2.0,0.273025,-0.407201
59998,0.0,2.0,1.0,2.0,130.0,144.0,2016,2.0,-0.564878,-0.407201


In [33]:
# Crear el modelo 
forest = RandomForestRegressor(n_estimators= 74, min_samples_split=10,min_samples_leaf=4,max_features= "sqrt",max_depth=42,bootstrap = False, verbose=2)
#Lo entreno 
forest.fit(x_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 74


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


building tree 2 of 74
building tree 3 of 74
building tree 4 of 74
building tree 5 of 74
building tree 6 of 74
building tree 7 of 74
building tree 8 of 74
building tree 9 of 74
building tree 10 of 74
building tree 11 of 74
building tree 12 of 74
building tree 13 of 74
building tree 14 of 74
building tree 15 of 74
building tree 16 of 74
building tree 17 of 74
building tree 18 of 74
building tree 19 of 74
building tree 20 of 74
building tree 21 of 74
building tree 22 of 74
building tree 23 of 74
building tree 24 of 74
building tree 25 of 74
building tree 26 of 74
building tree 27 of 74
building tree 28 of 74
building tree 29 of 74
building tree 30 of 74
building tree 31 of 74
building tree 32 of 74
building tree 33 of 74
building tree 34 of 74
building tree 35 of 74
building tree 36 of 74
building tree 37 of 74
building tree 38 of 74
building tree 39 of 74
building tree 40 of 74
building tree 41 of 74
building tree 42 of 74
building tree 43 of 74
building tree 44 of 74
building tree 45 of

[Parallel(n_jobs=1)]: Done  74 out of  74 | elapsed:   27.6s finished


RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=42,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=4, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=74,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=2, warm_start=False)

In [34]:
#Prediccion
prediccion =forest.predict(test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  74 out of  74 | elapsed:    1.2s finished


In [35]:
test_original = pd.read_csv("test.csv")
predicciones = test_original.loc[:, ['id','ciudad']]
predicciones.rename(columns = {'ciudad' : 'target'}, inplace = True)
predicciones['target'] = prediccion
predicciones = predicciones.set_index('id')

In [37]:
predicciones.to_csv(r'prediccionRF1.csv')